In [1]:
#TODO: stratified KFOLD???? +
#TODO: усреднее по фолдам ?
#TODO: параметры catboost +|- для категориальных признаков еще посомтреть
#TODO: кастомная метрика !!!


#TODO: идея 1: 13 катбустов независимых + catboost/усреднение/другая функция 
# 1 GPU 2 CPU 3 MIX
#TODO: идея 2: 13 катбустов зависимых каждый делает фичу и она подается другому на вход + catboost/усреднение/другая функция
# 1 GPU 2 CPU 3 MIX добавить веса на фактор старой модели
#TODO: объединение двух идей

#TODO: идея 3: сделать фичи и засунуть все целиком
#TODO: возможно с датами по другому

import pandas as pd 
path_test = r'C:\Users\pavlo\Desktop\Kaggle\AE\test_data.csv\test_data.csv'
#path_test = None
path_train = r'C:\Users\pavlo\Desktop\Kaggle\AE\train_data.csv\train_data.csv'
path_label = r'C:\Users\pavlo\Desktop\Kaggle\AE\train_labels.csv\train_labels.csv'

In [2]:
#train
import time
start = time.time()
chunk = pd.read_csv(path_train,chunksize=1000000)
pd_df = pd.concat(chunk)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")


Read csv with chunks:  105.59749054908752 sec


In [3]:
#target
import time
start = time.time()
chunk = pd.read_csv(path_label,chunksize=1000000)
dfl = pd.concat(chunk)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

Read csv with chunks:  0.37396955490112305 sec


In [4]:
#test
import time
start = time.time()
chunk = pd.read_csv(path_test,chunksize=1000000)
pd_df_test = pd.concat(chunk)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

Read csv with chunks:  219.54136896133423 sec


In [5]:
dfl

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1


In [6]:
# VERSION NAME FOR SAVED MODEL FILES
VER = 1

# TRAIN RANDOM SEED
SEED = 777

# FILL NAN VALUE
NAN_VALUE = -1000 # will fit in int8

# FOLDS PER MODEL
FOLD = 5
CATBOOST_COUNTER = 13

In [7]:
def func(x: str) -> int:
    return int(x[-16:], 16)
dfl['ID'] = dfl['customer_ID'].map(func)
target = dfl['target']


test = pd_df_test.fillna(NAN_VALUE)
test['ID'] = test['customer_ID'].map(func)
df_dict = test[['ID', 'customer_ID']].drop_duplicates() #словарь
test = test.drop(columns = ['customer_ID'] )

train = pd_df.fillna(NAN_VALUE)
train['ID'] = train['customer_ID'].map(func)
train = train.drop(columns = ['customer_ID'] )

#разделение на фичи
features = train.drop(['S_2'], axis=1).columns.to_list()
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87"
]
num_features = [col for col in features if col not in cat_features]

In [8]:
train

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,ID
0,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,-1000.0,...,-1000.0,-1000.0,0.002427,0.003706,0.003818,-1000.0,0.000569,0.000610,0.002674,13914591055249847850
1,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,-1000.0,...,-1000.0,-1000.0,0.003954,0.003167,0.005032,-1000.0,0.009576,0.005492,0.009217,13914591055249847850
2,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,-1000.0,...,-1000.0,-1000.0,0.003269,0.007329,0.000427,-1000.0,0.003429,0.006986,0.002603,13914591055249847850
3,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,-1000.0,...,-1000.0,-1000.0,0.006117,0.004516,0.003200,-1000.0,0.008419,0.006527,0.009600,13914591055249847850
4,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,-1000.0,...,-1000.0,-1000.0,0.003671,0.004946,0.008889,-1000.0,0.001670,0.008126,0.009827,13914591055249847850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,2017-11-05,0.979332,0.416013,0.020818,0.828199,0.003487,0.090743,0.005340,0.025139,-1000.0,...,-1000.0,-1000.0,0.006838,0.003680,0.000457,-1000.0,0.000905,0.001498,0.002774,10020895588212557441
5531447,2017-12-23,0.984907,0.296536,0.007209,0.812610,0.005904,0.079886,0.002243,0.023691,-1000.0,...,-1000.0,-1000.0,0.003310,0.007097,0.007857,-1000.0,0.002777,0.008225,0.008856,10020895588212557441
5531448,2018-01-06,0.983019,0.443984,0.013151,0.815422,0.003457,0.100503,0.002111,0.012343,-1000.0,...,-1000.0,-1000.0,0.009955,0.009994,0.001088,-1000.0,0.005693,0.006773,0.005566,10020895588212557441
5531449,2018-02-06,0.969861,0.442553,0.009855,1.003541,0.005117,0.101802,0.009930,0.008578,-1000.0,...,-1000.0,-1000.0,0.005541,0.006564,0.009883,-1000.0,0.008123,0.001168,0.003983,10020895588212557441


In [9]:
test

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,ID
0,2019-02-19,0.631315,0.001912,0.010728,0.814497,0.007547,0.168651,0.009971,0.002347,0.113189,...,-1000.0,-1000.0,-1000.000000,0.004669,-1000.000000,-1000.0,-1000.000000,0.008281,-1000.000000,8717704911770597815
1,2019-03-25,0.587042,0.005275,0.011026,0.810848,0.001817,0.241389,0.000166,0.009132,0.123035,...,-1000.0,-1000.0,0.000142,0.004940,0.009021,-1000.0,0.003695,0.003753,0.001460,8717704911770597815
2,2019-04-25,0.609056,0.003326,0.016390,1.004620,0.000114,0.266976,0.004196,0.004192,0.125319,...,-1000.0,-1000.0,0.000074,0.002114,0.004656,-1000.0,0.003155,0.002156,0.006482,8717704911770597815
3,2019-05-20,0.614911,0.009065,0.021672,0.816549,0.009722,0.188947,0.004123,0.015325,0.123439,...,-1000.0,-1000.0,0.004743,0.006392,0.002890,-1000.0,0.006044,0.005206,0.007855,8717704911770597815
4,2019-06-15,0.591673,0.238794,0.015923,0.810456,0.002026,0.180035,0.000731,0.011281,0.122212,...,-1000.0,-1000.0,0.008133,0.004329,0.008384,-1000.0,0.001008,0.007421,0.009471,8717704911770597815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11363757,2018-12-19,0.440834,0.000144,0.008638,0.818277,0.001797,-1000.000000,0.006421,0.002710,0.119515,...,-1000.0,-1000.0,0.006512,0.008873,0.001236,-1000.0,0.009655,0.005519,0.001351,16969632291170509645
11363758,2019-01-19,0.445590,0.006831,0.009636,0.815558,0.009876,-1000.000000,0.003611,0.006921,0.118299,...,-1000.0,-1000.0,0.003374,0.009006,0.004181,-1000.0,0.007839,0.000256,0.004916,16969632291170509645
11363759,2019-02-17,0.460082,0.005246,0.004420,0.818864,0.000665,-1000.000000,0.000559,0.007058,0.114429,...,-1000.0,-1000.0,0.007798,0.007739,0.008192,-1000.0,0.009386,0.003594,0.005603,16969632291170509645
11363760,2019-03-19,0.460331,0.002663,0.009529,0.816313,0.009027,-1000.000000,0.008527,0.006851,0.107068,...,-1000.0,-1000.0,0.007196,0.008456,0.004774,-1000.0,0.006198,0.005605,0.005972,16969632291170509645


In [10]:
dfl

,customer_ID,target,ID
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,13914591055249847850
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,11750091188498716901
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,7128959966677571777
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,6537921148391624412
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,2065103583825424365
...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0,1903558268166310647
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0,16552273430446277254
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0,17171417197321625590
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1,3187705857149035617


In [11]:
train_w_rnk = (
            train.assign(
                rn=train.sort_values(["S_2"], ascending=True)
                .groupby(["ID"])
                .cumcount()
                + 1
            )
            .sort_values(["ID", "S_2"])
        )

test_w_rnk = (
            test.assign(
                rn=test.sort_values(["S_2"], ascending=True)
                .groupby(["ID"])
                .cumcount()
                + 1
            )
            .sort_values(["ID", "S_2"])
        )

In [12]:
train_w_rnk

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,ID,rn
1758670,2017-03-17,-1000.000000,0.007430,0.000974,0.814975,0.006971,0.133906,0.002783,0.007320,-1000.0,...,-1000.0,0.003158,0.002059,0.002745,-1000.0,0.003904,0.005236,0.000674,3249127622875,1
1758671,2017-04-25,-1000.000000,0.030773,0.014857,1.009534,0.001177,0.131934,0.004325,0.014866,-1000.0,...,-1000.0,0.008519,0.005091,0.002355,-1000.0,0.005433,0.005165,0.002305,3249127622875,2
1758672,2017-05-09,-1000.000000,0.444905,0.011602,1.001185,0.006581,0.224990,0.004547,0.006408,-1000.0,...,-1000.0,0.007582,0.009359,0.000266,-1000.0,0.008928,0.004446,0.006803,3249127622875,3
1758673,2017-06-15,-1000.000000,0.621244,0.014671,1.002406,0.008547,0.094222,0.009643,0.001070,-1000.0,...,-1000.0,0.003126,0.005755,0.002874,-1000.0,0.008529,0.007402,0.006164,3249127622875,4
1758674,2017-07-25,-1000.000000,0.005819,0.006996,1.009531,0.007636,0.094472,0.003742,0.004317,-1000.0,...,-1000.0,0.008203,0.002731,0.008378,-1000.0,0.007053,0.000510,0.008520,3249127622875,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639475,2017-11-26,0.829680,0.007550,0.014158,0.812729,0.003634,0.172066,0.002341,0.017156,-1000.0,...,-1000.0,0.000489,0.004248,0.003267,-1000.0,0.002256,0.000626,0.002793,18446722875431722897,9
4639476,2017-12-08,0.831850,0.007073,0.123060,0.813213,0.002243,0.143518,0.009439,0.009797,-1000.0,...,-1000.0,0.000326,0.003504,0.001147,-1000.0,0.001454,0.003808,0.008920,18446722875431722897,10
4639477,2018-01-13,0.820854,0.005802,0.002709,1.007401,0.006969,0.158240,0.003882,0.011916,-1000.0,...,-1000.0,0.009697,0.002680,0.002823,-1000.0,0.009852,0.005884,0.005553,18446722875431722897,11
4639478,2018-02-10,0.826930,0.002578,0.002212,1.002295,0.005611,0.157103,0.006475,0.011516,-1000.0,...,-1000.0,0.009835,0.008028,0.001107,-1000.0,0.000823,0.003918,0.005617,18446722875431722897,12


In [13]:
test_w_rnk

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,ID,rn
5338738,2018-04-19,0.713401,0.000554,0.117755,0.817475,0.008335,0.329261,0.006741,0.000602,0.014962,...,-1000.0,0.009956,0.001119,0.002929,-1000.0,0.006310,0.009409,0.003384,63146778244,1
5338739,2018-05-10,0.709460,0.095031,0.012701,1.009518,0.004708,0.309152,0.008658,0.001585,0.008510,...,-1000.0,0.005422,0.001288,0.001381,-1000.0,0.003788,0.003687,0.008121,63146778244,2
5338740,2018-06-18,0.673798,0.359847,0.013161,1.008868,0.009521,0.320545,0.000610,0.003245,0.013809,...,-1000.0,0.000513,0.009868,0.008107,-1000.0,0.002734,0.009525,0.004993,63146778244,3
5338741,2018-07-02,0.672998,0.000021,0.014574,1.004637,0.008925,0.324475,0.002860,0.005429,0.009821,...,-1000.0,0.005494,0.006410,0.007029,-1000.0,0.009496,0.004844,0.003215,63146778244,4
5338742,2018-08-08,0.683888,0.036531,0.006392,1.004565,0.008844,0.337125,0.008048,0.001897,0.014357,...,-1000.0,0.007323,0.005917,0.004211,-1000.0,0.008838,0.004783,0.009327,63146778244,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777503,2019-06-17,0.858884,0.005195,0.035609,1.009608,0.006212,0.088389,0.009433,0.001570,-1000.000000,...,-1000.0,0.004329,0.002520,0.001484,-1000.0,0.008038,0.000924,0.004472,18446738444797682302,9
1777504,2019-07-18,0.869829,0.003248,0.021662,0.817368,0.008757,0.094277,0.002790,0.009214,-1000.000000,...,-1000.0,0.000181,0.004407,0.003041,-1000.0,0.000639,0.002023,0.003441,18446738444797682302,10
1777505,2019-08-26,0.842224,0.209636,0.019366,1.008697,0.004435,0.101578,0.004859,0.010474,-1000.000000,...,-1000.0,0.003742,0.008515,0.000113,-1000.0,0.001874,0.007644,0.009191,18446738444797682302,11
1777506,2019-09-01,0.841814,0.390544,0.018823,1.001184,0.003604,0.096814,0.000962,0.010545,-1000.000000,...,-1000.0,0.006875,0.002100,0.001578,-1000.0,0.002781,0.007635,0.004580,18446738444797682302,12


In [ ]:
t_train =train_w_rnk.loc[train_w_rnk.rn == 1].merge(train_w_rnk.loc[train_w_rnk.rn == 2], how = 'left', on = ['ID'])
t_train

In [14]:
class AmexMetric:
    
    @staticmethod
    #крутая мертика
    def amex_metric_mod(y_true, y_pred):
        #y_pred = expit(y_pred).astype(int)
        #y_true = y_true.astype(int)
        labels     = np.transpose(np.array([y_true, y_pred]))
        labels     = labels[labels[:, 1].argsort()[::-1]]
        weights    = np.where(labels[:,0]==0, 20, 1)
        cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
        top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

        gini = [0,0]
        for i in [1,0]:
            labels         = np.transpose(np.array([y_true, y_pred]))
            labels         = labels[labels[:, i].argsort()[::-1]]
            weight         = np.where(labels[:,0]==0, 20, 1)
            weight_random  = np.cumsum(weight / np.sum(weight))
            total_pos      = np.sum(labels[:, 0] *  weight)
            cum_pos_found  = np.cumsum(labels[:, 0] * weight)
            lorentz        = cum_pos_found / total_pos
            gini[i]        = np.sum((lorentz - weight_random) * weight)

        return 0.5 * (gini[1]/gini[0] + top_four)
    
    def is_max_optimal(self):
        return True # greater is better 

    def evaluate(self, approxes, target, weight):            
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target).astype(int)
        approx = approxes[0]
        score = self.amex_metric_mod(y_true, approx)
        return score, 1

    def get_final_error(self, error, weight):
        return error

In [29]:
VER = 2

In [15]:
import catboost as cb
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from paramsearch import paramsearch
from itertools import product,chain


def crossvaltest(params,train_set,train_label,cat_dims,n_splits, random_state):
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state = random_state) 
    res = []
    for train_index, test_index in kf.split(train_set, train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]
        labels = train_label[train_index]
        test_labels = train_label[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)
        
        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    
    return np.mean(res)

def catboost_param_tune(params,train_set,train_label,n_splits, random_state, cat_dims=None):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain( ps.grid_search(['border_count']),
                      ps.grid_search(['bootstrap_type']),
                      #ps.grid_search(['bagging_temperature']),
                      ps.grid_search(['random_strength']),
                      ps.grid_search(['max_ctr_complexity']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        print(prms)
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits, random_state)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()

#bestparams = catboost_param_tune(params,X_train,y_train,cat_dims)

In [30]:
print(VER)

2


In [31]:
def meta_factors_get(params, X_train, y_train, n_splits, random_state, cat_dims):
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state = random_state) 
    pred = pd.DataFrame()
    kfold = 0
    for train_index, test_index in kf.split(X_train, y_train):
        kfold = kfold + 1
        train = X_train.iloc[train_index,:]
        test =X_train.iloc[test_index,:]
        labels = y_train[train_index]
        test_labels = y_train[test_index]
        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)
        predict = pd.DataFrame({str(kfold): clf.predict_proba(X_train)[:, 1].tolist(),})
        pred = pd.concat([pred, predict], axis = 1)   
        #print(pred)
        #todo: веса
    return pred.mean(axis = 1)


def func_calculate(train_w_rnk, test_w_rnk, dfl, catboost_counter, ver, n_splits, random_state, category_cols):
    #dfl - target
    #print(cat_features)
    #print(cat_dims)
    #print(train_w_rnk.iloc[:, cat_dims])
    for header in category_cols:
        train_w_rnk[header] = train_w_rnk[header].astype('category').cat.codes
        test_w_rnk[header] = test_w_rnk[header].astype('category').cat.codes
        
    params = {'task_type': 'GPU',
              'depth':[1,2,3,4,5,6,7,8,9,10], #глубина дерева
              'iterations':[1500,2000, 2500], # количество итераций 
              'learning_rate':[0.001,0.01, 0.03, 0.15,0.2,0.25], #скорость обученя
              'l2_leaf_reg':[1,3,5,10,100], #коэфициент в регуляризации
              'bootstrap_type':['Bayesian'], #как выбираем элементы  из выборки  #'Bernoulli', 'No' #нужно убрать temperature
              'border_count':[5,10,20,50,100,200],#скорость обучения GPU
                  'bagging_temperature':[0.03,0.09,0.25,0.75], # коэффициенты с помощью которых ге
                    #генерируются веса в баесовском бутстрепе 
                  'random_strength':[0.2,0.5,0.8], #коэффициент участвующий при разделения дерева и оценки подсчета очков
                  'max_ctr_complexity':[1,2,3,4,5], # количество листьев для категориальных признаков поиск переобучения 
              'thread_count': 4000, #4352 количество ядер на GPU
              'metric_period': 50, #каждые 50 итерацией вычисляется кастомная метрика
               'eval_metric': ['Logloss'], #AmexMetric() 
               'early_stopping_rounds': 50 #остановится если переобучилсиь
              }    

    
    #test_blend = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID']
    test_stack = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID'].to_frame()
    
    #train_blend = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID']
    train_stack = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID'].to_frame()
    
    for rank in range(1, catboost_counter+1):
        dft =  train_w_rnk[train_w_rnk.rn == rank]
        train_id = dft['ID']
        data = dft.merge(dfl, how = 'inner', on = 'ID')
        X_train = data.drop(columns = ['ID', 'S_2', 'target', 'rn', 'customer_ID'])
        y_train = data['target']
        cat_dims = [X_train.columns.get_loc(i) for i in category_cols[:-1]]
        bestparams = catboost_param_tune(params,X_train,y_train,n_splits, random_state, cat_dims)

        dftt =  test_w_rnk[test_w_rnk.rn == rank]
        test_id = dftt['ID']
        X_test = dftt.drop(columns = ['ID', 'S_2', 'rn', ])

        #stack
        df_meta = meta_factors_get(bestparams, X_train, y_train, n_splits, random_state, cat_dims)
        df_meta = df_meta.to_frame().rename(columns = {0: str(rank)})
        df_train= pd.concat([train_id.reset_index(drop = True), df_meta], axis = 1)
        train_stack = train_stack.merge(df_train, how = 'left', on = ['ID'])
        
        clf = cb.CatBoostClassifier(**bestparams)
        print(train_stack)
      
        clf.fit(X_train, np.ravel(y_train), cat_features=cat_dims)
        clf.save_model(str(VER)+"_"+str(rank)+"_clf")
        print(str(VER)+"_"+str(rank)+': save model')
        res = clf.predict_proba(X_test)
        pred = pd.DataFrame({str(rank): res[:, 1].tolist(),})
        df_test = pd.concat([test_id.reset_index(drop = True), pred], axis = 1)
        test_stack = test_stack.merge(df_test, how = 'left', on = ['ID'])
        print(test_stack)
        test_stack.to_excel(str(VER)+'catboost_test.xlsx', index = False)
        train_stack.to_excel(str(VER)+'catboost_train.xlsx', index = False)
        print(str(VER)+"_"+str(rank)+': save result')
    return train_stack, test_stack 
      
train_stack, test_stack = func_calculate(train_w_rnk, test_w_rnk, dfl, CATBOOST_COUNTER, VER, FOLD, SEED, cat_features)

0:	learn: 0.6925373	total: 81.7ms	remaining: 40.8s
50:	learn: 0.6638731	total: 609ms	remaining: 5.36s
100:	learn: 0.6377029	total: 978ms	remaining: 3.86s
150:	learn: 0.6132421	total: 1.32s	remaining: 3.05s
200:	learn: 0.5916385	total: 1.66s	remaining: 2.46s
250:	learn: 0.5724871	total: 2s	remaining: 1.98s
300:	learn: 0.5555869	total: 2.34s	remaining: 1.55s
350:	learn: 0.5406713	total: 2.69s	remaining: 1.14s
400:	learn: 0.5284648	total: 3.04s	remaining: 750ms
450:	learn: 0.5185172	total: 3.38s	remaining: 367ms
499:	learn: 0.5095435	total: 3.71s	remaining: 0us
0:	learn: 0.6925370	total: 180ms	remaining: 1m 29s
50:	learn: 0.6638359	total: 555ms	remaining: 4.88s
100:	learn: 0.6379397	total: 897ms	remaining: 3.54s
150:	learn: 0.6134415	total: 1.24s	remaining: 2.86s
200:	learn: 0.5917958	total: 1.58s	remaining: 2.35s
250:	learn: 0.5726073	total: 1.93s	remaining: 1.91s
300:	learn: 0.5556753	total: 2.27s	remaining: 1.5s
350:	learn: 0.5407336	total: 2.61s	remaining: 1.11s
400:	learn: 0.5284022	

300:	learn: 0.5557306	total: 2.37s	remaining: 1.57s
350:	learn: 0.5410372	total: 2.75s	remaining: 1.17s
400:	learn: 0.5282831	total: 3.16s	remaining: 780ms
450:	learn: 0.5182256	total: 3.52s	remaining: 382ms
499:	learn: 0.5092479	total: 3.87s	remaining: 0us
0:	learn: 0.6924933	total: 70.7ms	remaining: 35.3s
50:	learn: 0.6621200	total: 733ms	remaining: 6.45s
100:	learn: 0.6354295	total: 1.1s	remaining: 4.37s
150:	learn: 0.6116774	total: 1.48s	remaining: 3.42s
200:	learn: 0.5906470	total: 1.89s	remaining: 2.81s
250:	learn: 0.5721461	total: 2.3s	remaining: 2.29s
300:	learn: 0.5556753	total: 2.7s	remaining: 1.78s
350:	learn: 0.5412809	total: 3.1s	remaining: 1.31s
400:	learn: 0.5283694	total: 3.49s	remaining: 862ms
450:	learn: 0.5182671	total: 3.89s	remaining: 422ms
499:	learn: 0.5091908	total: 4.28s	remaining: 0us
0:	learn: 0.6925058	total: 75.8ms	remaining: 37.8s
50:	learn: 0.6623265	total: 741ms	remaining: 6.52s
100:	learn: 0.6355615	total: 1.13s	remaining: 4.45s
150:	learn: 0.6117709	to

50:	learn: 0.6638355	total: 632ms	remaining: 5.56s
100:	learn: 0.6378379	total: 977ms	remaining: 3.86s
150:	learn: 0.6133407	total: 1.31s	remaining: 3.04s
200:	learn: 0.5916195	total: 1.65s	remaining: 2.46s
250:	learn: 0.5725273	total: 1.99s	remaining: 1.97s
300:	learn: 0.5555994	total: 2.33s	remaining: 1.54s
350:	learn: 0.5406610	total: 2.67s	remaining: 1.13s
400:	learn: 0.5284378	total: 3s	remaining: 742ms
450:	learn: 0.5184816	total: 3.34s	remaining: 363ms
499:	learn: 0.5094420	total: 3.67s	remaining: 0us
0:	learn: 0.6925373	total: 73.6ms	remaining: 36.7s
50:	learn: 0.6638732	total: 596ms	remaining: 5.25s
100:	learn: 0.6377028	total: 939ms	remaining: 3.71s
150:	learn: 0.6132422	total: 1.28s	remaining: 2.95s
200:	learn: 0.5916384	total: 1.62s	remaining: 2.41s
250:	learn: 0.5724871	total: 1.96s	remaining: 1.95s
300:	learn: 0.5555869	total: 2.3s	remaining: 1.52s
350:	learn: 0.5406708	total: 2.64s	remaining: 1.12s
400:	learn: 0.5285263	total: 2.98s	remaining: 735ms
450:	learn: 0.5184643

350:	learn: 0.5407339	total: 2.88s	remaining: 1.22s
400:	learn: 0.5285823	total: 3.22s	remaining: 794ms
450:	learn: 0.5186318	total: 3.6s	remaining: 391ms
499:	learn: 0.5096984	total: 3.94s	remaining: 0us
0:	learn: 0.6925369	total: 304ms	remaining: 2m 31s
50:	learn: 0.6638354	total: 780ms	remaining: 6.87s
100:	learn: 0.6378380	total: 1.2s	remaining: 4.76s
150:	learn: 0.6133417	total: 1.65s	remaining: 3.81s
200:	learn: 0.5916199	total: 2.1s	remaining: 3.12s
250:	learn: 0.5725281	total: 2.52s	remaining: 2.5s
300:	learn: 0.5556002	total: 2.91s	remaining: 1.92s
350:	learn: 0.5406615	total: 3.29s	remaining: 1.4s
400:	learn: 0.5285580	total: 3.67s	remaining: 905ms
450:	learn: 0.5185333	total: 4.1s	remaining: 445ms
499:	learn: 0.5095454	total: 4.49s	remaining: 0us
0:	learn: 0.6925374	total: 238ms	remaining: 1m 58s
50:	learn: 0.6638732	total: 644ms	remaining: 5.67s
100:	learn: 0.6377029	total: 1.02s	remaining: 4.05s
150:	learn: 0.6132421	total: 1.41s	remaining: 3.25s
200:	learn: 0.5916384	tota

100:	learn: 0.6379985	total: 930ms	remaining: 3.67s
150:	learn: 0.6133997	total: 1.25s	remaining: 2.9s
200:	learn: 0.5917639	total: 1.58s	remaining: 2.35s
250:	learn: 0.5725871	total: 1.91s	remaining: 1.89s
300:	learn: 0.5557387	total: 2.23s	remaining: 1.47s
350:	learn: 0.5407341	total: 2.55s	remaining: 1.08s
400:	learn: 0.5284065	total: 2.88s	remaining: 710ms
450:	learn: 0.5184683	total: 3.2s	remaining: 348ms
499:	learn: 0.5095446	total: 3.52s	remaining: 0us
0:	learn: 0.6925367	total: 87.5ms	remaining: 43.7s
50:	learn: 0.6638353	total: 657ms	remaining: 5.79s
100:	learn: 0.6378384	total: 989ms	remaining: 3.91s
150:	learn: 0.6132509	total: 1.32s	remaining: 3.04s
200:	learn: 0.5916205	total: 1.65s	remaining: 2.46s
250:	learn: 0.5725269	total: 1.98s	remaining: 1.96s
300:	learn: 0.5556000	total: 2.31s	remaining: 1.53s
350:	learn: 0.5406611	total: 2.65s	remaining: 1.12s
400:	learn: 0.5283809	total: 2.98s	remaining: 735ms
450:	learn: 0.5183679	total: 3.31s	remaining: 360ms
499:	learn: 0.5094

...

600:	learn: 0.0986597	total: 14s	remaining: 20.9s
650:	learn: 0.0930652	total: 15.2s	remaining: 19.8s
700:	learn: 0.0879883	total: 16.3s	remaining: 18.6s
750:	learn: 0.0832842	total: 17.4s	remaining: 17.4s
800:	learn: 0.0784637	total: 18.6s	remaining: 16.2s
850:	learn: 0.0742981	total: 19.7s	remaining: 15.1s
900:	learn: 0.0703784	total: 20.9s	remaining: 13.9s
950:	learn: 0.0666589	total: 22s	remaining: 12.7s
1000:	learn: 0.0630540	total: 23.2s	remaining: 11.6s
1050:	learn: 0.0596805	total: 24.3s	remaining: 10.4s
1100:	learn: 0.0566281	total: 25.5s	remaining: 9.23s
1150:	learn: 0.0537967	total: 26.6s	remaining: 8.07s
1200:	learn: 0.0510346	total: 27.8s	remaining: 6.91s
1250:	learn: 0.0483392	total: 28.9s	remaining: 5.75s
1300:	learn: 0.0457252	total: 30.1s	remaining: 4.6s
1350:	learn: 0.0433678	total: 31.2s	remaining: 3.44s
1400:	learn: 0.0410688	total: 32.4s	remaining: 2.29s
1450:	learn: 0.0388182	total: 33.5s	remaining: 1.13s
1499:	learn: 0.0367966	total: 34.6s	remaining: 0us
0:	learn

750:	learn: 0.0508686	total: 26.2s	remaining: 26.1s
800:	learn: 0.0467641	total: 27.9s	remaining: 24.3s
850:	learn: 0.0430370	total: 29.6s	remaining: 22.6s
900:	learn: 0.0396290	total: 31.3s	remaining: 20.8s
950:	learn: 0.0364531	total: 33.1s	remaining: 19.1s
1000:	learn: 0.0337393	total: 34.8s	remaining: 17.3s
1050:	learn: 0.0310465	total: 36.5s	remaining: 15.6s
1100:	learn: 0.0285427	total: 38.2s	remaining: 13.9s
1150:	learn: 0.0264462	total: 40s	remaining: 12.1s
1200:	learn: 0.0243543	total: 41.7s	remaining: 10.4s
1250:	learn: 0.0224322	total: 43.4s	remaining: 8.64s
1300:	learn: 0.0206572	total: 45.1s	remaining: 6.91s
1350:	learn: 0.0192284	total: 46.9s	remaining: 5.17s
1400:	learn: 0.0176740	total: 48.6s	remaining: 3.43s
1450:	learn: 0.0164053	total: 50.3s	remaining: 1.7s
1499:	learn: 0.0153035	total: 52s	remaining: 0us
0:	learn: 0.4138367	total: 276ms	remaining: 6m 54s
50:	learn: 0.1857352	total: 2.1s	remaining: 59.8s
100:	learn: 0.1669393	total: 3.83s	remaining: 53.1s
150:	learn:

900:	learn: 0.0174467	total: 52.1s	remaining: 34.6s
950:	learn: 0.0154171	total: 55s	remaining: 31.7s
1000:	learn: 0.0136896	total: 57.8s	remaining: 28.8s
1050:	learn: 0.0121166	total: 1m	remaining: 25.9s
1100:	learn: 0.0108439	total: 1m 3s	remaining: 23s
1150:	learn: 0.0097406	total: 1m 6s	remaining: 20.1s
1200:	learn: 0.0087731	total: 1m 9s	remaining: 17.3s
1250:	learn: 0.0078295	total: 1m 12s	remaining: 14.4s
1300:	learn: 0.0070629	total: 1m 15s	remaining: 11.5s
1350:	learn: 0.0064233	total: 1m 17s	remaining: 8.6s
1400:	learn: 0.0058739	total: 1m 20s	remaining: 5.71s
1450:	learn: 0.0052184	total: 1m 23s	remaining: 2.83s
1499:	learn: 0.0047772	total: 1m 26s	remaining: 0us
0:	learn: 0.4115596	total: 300ms	remaining: 7m 30s
50:	learn: 0.1748603	total: 3.22s	remaining: 1m 31s
100:	learn: 0.1483633	total: 6.1s	remaining: 1m 24s
150:	learn: 0.1271321	total: 9s	remaining: 1m 20s
200:	learn: 0.1094806	total: 11.9s	remaining: 1m 16s
250:	learn: 0.0944605	total: 14.7s	remaining: 1m 13s
300:	l

1050:	learn: 0.1916320	total: 8.36s	remaining: 3.57s
1100:	learn: 0.1911823	total: 8.75s	remaining: 3.17s
1150:	learn: 0.1907438	total: 9.13s	remaining: 2.77s
1200:	learn: 0.1903133	total: 9.52s	remaining: 2.37s
1250:	learn: 0.1898874	total: 9.9s	remaining: 1.97s
1300:	learn: 0.1894889	total: 10.3s	remaining: 1.57s
1350:	learn: 0.1890722	total: 10.7s	remaining: 1.18s
1400:	learn: 0.1886909	total: 11s	remaining: 781ms
1450:	learn: 0.1883148	total: 11.4s	remaining: 386ms
1499:	learn: 0.1879248	total: 11.8s	remaining: 0us
0:	learn: 0.4658611	total: 90.3ms	remaining: 2m 15s
50:	learn: 0.2150370	total: 734ms	remaining: 20.9s
100:	learn: 0.2084517	total: 1.12s	remaining: 15.5s
150:	learn: 0.2052979	total: 1.5s	remaining: 13.4s
200:	learn: 0.2033131	total: 1.88s	remaining: 12.2s
250:	learn: 0.2018667	total: 2.27s	remaining: 11.3s
300:	learn: 0.2006729	total: 2.65s	remaining: 10.6s
350:	learn: 0.1995983	total: 3.03s	remaining: 9.92s
400:	learn: 0.1987677	total: 3.41s	remaining: 9.35s
450:	lear

0:	learn: 0.4666808	total: 86.7ms	remaining: 2m 9s
50:	learn: 0.2156670	total: 681ms	remaining: 19.4s
100:	learn: 0.2087269	total: 1.09s	remaining: 15.1s
150:	learn: 0.2055526	total: 1.5s	remaining: 13.4s
200:	learn: 0.2036767	total: 1.92s	remaining: 12.4s
250:	learn: 0.2022026	total: 2.33s	remaining: 11.6s
300:	learn: 0.2011358	total: 2.74s	remaining: 10.9s
350:	learn: 0.2002406	total: 3.15s	remaining: 10.3s
400:	learn: 0.1993837	total: 3.56s	remaining: 9.77s
450:	learn: 0.1986594	total: 3.98s	remaining: 9.25s
500:	learn: 0.1980217	total: 4.41s	remaining: 8.79s
550:	learn: 0.1974053	total: 4.82s	remaining: 8.3s
600:	learn: 0.1968659	total: 5.23s	remaining: 7.82s
650:	learn: 0.1963509	total: 5.64s	remaining: 7.36s
700:	learn: 0.1958242	total: 6.05s	remaining: 6.9s
750:	learn: 0.1953341	total: 6.46s	remaining: 6.45s
800:	learn: 0.1948604	total: 6.88s	remaining: 6s
850:	learn: 0.1944290	total: 7.29s	remaining: 5.56s
900:	learn: 0.1940238	total: 7.7s	remaining: 5.12s
950:	learn: 0.1936078

In [23]:
import numpy as np
VER = 4
#l_CATBOOST = [(1,2), (3,4), (5,6), (7,8), (9,10), (11, 12), (12,13), (1,3), (2,4), (5, 7), (6, 8), (9,11), (10,12)]
#k = 2

l_CATBOOST = [(1,2,3,4,5,6,7,8,9,10,11,12,13)]
k = 13
CATBOOST_COUNTER = len(l_CATBOOST)
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87",
    #"D_62"#add
]
CATBOOST_COUNTER

1

In [ ]:
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87",
     #"D_62"#
]

def meta_factors_get(params, X_train, y_train, n_splits, random_state, cat_dims):
    kf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state = random_state) 
    pred = pd.DataFrame()
    kfold = 0
    for train_index, test_index in kf.split(X_train, y_train):
        kfold = kfold + 1
        train = X_train.iloc[train_index,:]
        test =X_train.iloc[test_index,:]
        labels = y_train[train_index]
        test_labels = y_train[test_index]
        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)
        predict = pd.DataFrame({str(kfold): clf.predict_proba(X_train)[:, 1].tolist(),})
        pred = pd.concat([pred, predict], axis = 1)   
        #print(pred)
        #todo: веса
    return pred.mean(axis = 1)

def func_calculate_v2(train_w_rnk, test_w_rnk, dfl, catboost_counter, ver, n_splits, random_state, category_cols, l_catboost,k):

     #params = {'task_type': 'GPU',
     #         'depth':[1,2,3,4,5,7,8,10], #глубина дерева
     #         'iterations':[1500,2000, 2500], # количество итераций 
     #         'learning_rate':[0.001, 0.01, 0.03,0.1,0.25, 0.5], #скорость обученя
     #         'l2_leaf_reg':[1,3,5,10,100], #коэфициент в регуляризации
     #         'bootstrap_type':['Bernoulli'], #как выбираем элементы  из выборки  #'Bernoulli', 'No' #нужно убрать temperature
     #         'border_count':[5,10,20,50,100,200],#скорость обучения GPU
     #             #'bagging_temperature':[0.03,0.25,0.75, 0.9], # коэффициенты с помощью которых ге
     #               #генерируются веса в баесовском бутстрепе 
     #             'random_strength':[0.2,0.5,0.8], #коэффициент участвующий при разделения дерева и оценки подсчета очков
     #             'max_ctr_complexity':[1,2,3,4,5], # количество листьев для категориальных признаков поиск переобучения 
     #         'thread_count': 4000, #4352 количество ядер на GPU
     #         'metric_period': 50, #каждые 50 итерацией вычисляется кастомная метрика
     #          'eval_metric': ['Logloss'], #AmexMetric() 
     #          'early_stopping_rounds': 50 #остановится если переобучилсиь
     #         }    
    params =  {'random_strength': [0.2],
    'border_count': [200],
    'bootstrap_type': ['Bernoulli'],
    'eval_metric': [AmexMetric()],
    'od_wait': [50],
    'iterations': [2500],
    'thread_count': [4000],
    'max_ctr_complexity': [1],
    'l2_leaf_reg': [0.95, 1,1.25],
    #'loss_function': 'Logloss',
    'task_type': ['CPU'],
    'depth': [2],
    'metric_period': [50],
    'learning_rate': [0.25, 0.24, 0.23]}


    
    #test_blend = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID']
    test_stack = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID'].to_frame()
    
    #train_blend = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID']
    train_stack = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID'].to_frame()
    
    l_drop = ['S_2', 'rn']
    #l_0 = ['ID', 'target', 'customer_ID']
    
    l_drop_test = ['S_2', 'rn']
    #l_0_test = ['ID']
    #category_cols_all = []
    
    for rank,c in enumerate(l_catboost):
        l_0 = ['ID', 'target', 'customer_ID']
        l_0_test = ['ID']
        category_cols_all = [x+'_'+str(c[0]) for j1,x in enumerate(category_cols)]
        #category_cols_all = []
        dtrain = train_w_rnk[train_w_rnk.rn == c[0]]
        #print(dtrain.columns)
        dtrain.columns = [x+'_'+str(c[0]) for j,x in enumerate(dtrain.columns)]
        dtrain =  dtrain.rename(columns = {'ID_'+str(c[0]):'ID'})
        l_0.extend([x+'_'+str(c[0]) for j,x in enumerate(l_drop)])
        l_drop_all = l_0
        
        for j in range(1, k):
            dtrain_next = train_w_rnk[train_w_rnk.rn == c[j]]
            dtrain_next.columns = [x+'_'+str(c[j]) for j1,x in enumerate(dtrain_next.columns)]
            
            l_drop_add = [x+'_'+str(c[j]) for j1,x in enumerate(l_drop)]
            #print(l_drop_res)
            #print(l_drop_add)
            l_drop_all.extend(l_drop_add)
            
            category_cols_add = [x+'_'+str(c[j]) for j1,x in enumerate(category_cols)]
            category_cols_all.extend(category_cols_add)
            dtrain_next =  dtrain_next.rename(columns = {'ID_'+str(c[j]):'ID'})
            dtrain = dtrain.merge(dtrain_next, how = 'left', on = ['ID'])
        dft = dtrain     
        train_id = dft['ID']
     
        data = dft.merge(dfl, how = 'inner', on = 'ID')
        #dtrain =  dtrain.rename(columns = {'ID':'ID_1'})
        
        print(data.columns)
    
        X_train = data.drop(columns = list(l_drop_all)).fillna(-1000)
        y_train = data['target']
        
        #print(list(X_train.columns))
        #print(list(X_train.columns)[51])
        cat_dims = [X_train.columns.get_loc(i) for i in category_cols_all[:-1]]
        #print(category_cols_all)
        #print(cat_dims)
        
        for header in category_cols_all:
             X_train[header] = X_train[header].astype('category').cat.codes

        bestparams = catboost_param_tune(params,X_train,y_train,n_splits, random_state, cat_dims)
        print(bestparams)
        
        #stack
        df_meta = meta_factors_get(bestparams, X_train, y_train, n_splits, random_state, cat_dims)
        df_meta = df_meta.to_frame().rename(columns = {0: str(rank)})
        df_train= pd.concat([train_id.reset_index(drop = True), df_meta], axis = 1)
        train_stack = train_stack.merge(df_train, how = 'left', on = ['ID'])
        clf = cb.CatBoostClassifier(**bestparams)
        print(train_stack)
      
        clf.fit(X_train, np.ravel(y_train), cat_features=cat_dims)
        clf.save_model(str(VER)+"_"+str(rank)+"_clf")
        print(str(VER)+"_"+str(rank)+': save model')
        
        del X_train

        dtest= test_w_rnk[test_w_rnk.rn == c[0]]
        dtest.columns = [x+'_'+str(c[0]) for j,x in enumerate(dtest.columns)]
        dtest =  dtest.rename(columns = {'ID_'+str(c[0]):'ID'})
        l_0_test.extend([x+'_'+str(c[0]) for j,x in enumerate(l_drop_test)])
        l_drop_all = l_0_test
        for j in range(1, k):
            dtest_next = test_w_rnk[test_w_rnk.rn == c[j]]
            dtest_next.columns = [x+'_'+str(c[j]) for j1,x in enumerate(dtest_next.columns)]
            
            l_drop_add = [x+'_'+str(c[j]) for j1,x in enumerate(l_drop_test)]
            l_drop_all.extend(l_drop_add)
            
            dtest_next =  dtest_next.rename(columns = {'ID_'+str(c[j]):'ID'})
            dtest = dtest.merge(dtest_next, how = 'left', on = ['ID'])
        
        
        dftt = dtest
        test_id = dftt['ID']
        #dtest =  dtest.rename(columns = {'ID':'ID_1'})
        X_test = dftt.drop(columns = list(l_drop_all)).fillna(-1000)
        for header in category_cols_all:
             X_test[header] = X_test[header].astype('category').cat.codes

        res = clf.predict_proba(X_test)
        pred = pd.DataFrame({str(rank): res[:, 1].tolist(),})
        df_test = pd.concat([test_id.reset_index(drop = True), pred], axis = 1)
        test_stack = test_stack.merge(df_test, how = 'left', on = ['ID'])
        print(test_stack)
        test_stack.to_excel(str(VER)+'catboost_test.xlsx', index = False)
        train_stack.to_excel(str(VER)+'catboost_train.xlsx', index = False)
        print(str(VER)+"_"+str(rank)+': save result')
    return train_stack, test_stack 
      
train_stack, test_stack = func_calculate_v2(train_w_rnk, test_w_rnk, dfl, CATBOOST_COUNTER,
                                            VER, FOLD, SEED, cat_features, l_CATBOOST, k)

Index(['S_2_1', 'P_2_1', 'D_39_1', 'B_1_1', 'B_2_1', 'R_1_1', 'S_3_1',
       'D_41_1', 'B_3_1', 'D_42_1',
       ...
       'D_139_13', 'D_140_13', 'D_141_13', 'D_142_13', 'D_143_13', 'D_144_13',
       'D_145_13', 'rn_13', 'customer_ID', 'target'],
      dtype='object', length=2473)
{'random_strength': 0.2, 'border_count': 200, 'bootstrap_type': 'Bernoulli', 'eval_metric': <__main__.AmexMetric object at 0x000001B930890A60>, 'od_wait': 50, 'iterations': 2500, 'thread_count': 4000, 'max_ctr_complexity': 1, 'l2_leaf_reg': 1, 'task_type': 'CPU', 'depth': 2, 'metric_period': 50, 'learning_rate': 0.25}


C:\Users\pavlo\anaconda3\lib\site-packages\catboost\core.py:2194: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.5114077	total: 1.06s	remaining: 44m 17s
50:	learn: 0.7320609	total: 13.9s	remaining: 11m 6s
100:	learn: 0.7488697	total: 25.9s	remaining: 10m 14s
150:	learn: 0.7568792	total: 37.8s	remaining: 9m 47s
200:	learn: 0.7627595	total: 49.7s	remaining: 9m 28s
250:	learn: 0.7669125	total: 1m 1s	remaining: 9m 13s
300:	learn: 0.7700502	total: 1m 13s	remaining: 8m 57s
350:	learn: 0.7733229	total: 1m 25s	remaining: 8m 44s
400:	learn: 0.7763870	total: 1m 37s	remaining: 8m 30s
450:	learn: 0.7783185	total: 1m 49s	remaining: 8m 15s
500:	learn: 0.7801814	total: 2m	remaining: 8m 1s
550:	learn: 0.7820898	total: 2m 12s	remaining: 7m 48s
600:	learn: 0.7838770	total: 2m 23s	remaining: 7m 34s
650:	learn: 0.7856939	total: 2m 35s	remaining: 7m 22s
700:	learn: 0.7870780	total: 2m 47s	remaining: 7m 8s
750:	learn: 0.7881525	total: 2m 58s	remaining: 6m 56s
800:	learn: 0.7894192	total: 3m 10s	remaining: 6m 44s
850:	learn: 0.7903730	total: 3m 22s	remaining: 6m 31s
900:	learn: 0.7915281	total: 3m 33s	remai

C:\Users\pavlo\anaconda3\lib\site-packages\catboost\core.py:2194: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.5173033	total: 626ms	remaining: 26m 3s
50:	learn: 0.7300081	total: 14s	remaining: 11m 10s
100:	learn: 0.7461555	total: 27.3s	remaining: 10m 49s
150:	learn: 0.7550779	total: 40.3s	remaining: 10m 27s
200:	learn: 0.7611214	total: 52.9s	remaining: 10m 4s
250:	learn: 0.7655206	total: 1m 5s	remaining: 9m 46s
300:	learn: 0.7691898	total: 1m 17s	remaining: 9m 29s
350:	learn: 0.7715852	total: 1m 30s	remaining: 9m 16s
400:	learn: 0.7739770	total: 1m 43s	remaining: 9m 2s
450:	learn: 0.7761730	total: 1m 55s	remaining: 8m 46s
500:	learn: 0.7776425	total: 2m 8s	remaining: 8m 32s
550:	learn: 0.7802161	total: 2m 20s	remaining: 8m 18s
600:	learn: 0.7817431	total: 2m 33s	remaining: 8m 4s
650:	learn: 0.7835077	total: 2m 45s	remaining: 7m 51s
700:	learn: 0.7845156	total: 2m 58s	remaining: 7m 37s
750:	learn: 0.7855632	total: 3m 10s	remaining: 7m 23s
800:	learn: 0.7866239	total: 3m 22s	remaining: 7m 10s
850:	learn: 0.7882049	total: 3m 35s	remaining: 6m 57s
900:	learn: 0.7889087	total: 3m 47s	rem

...

0:	learn: 0.5173033	total: 611ms	remaining: 25m 26s
50:	learn: 0.7279590	total: 14.1s	remaining: 11m 15s
100:	learn: 0.7447380	total: 26.6s	remaining: 10m 32s
150:	learn: 0.7530558	total: 39.2s	remaining: 10m 9s
200:	learn: 0.7591639	total: 51.9s	remaining: 9m 53s
250:	learn: 0.7639729	total: 1m 4s	remaining: 9m 36s
300:	learn: 0.7674905	total: 1m 16s	remaining: 9m 21s
350:	learn: 0.7701773	total: 1m 29s	remaining: 9m 7s
400:	learn: 0.7730875	total: 1m 42s	remaining: 8m 54s
450:	learn: 0.7755111	total: 1m 54s	remaining: 8m 39s
500:	learn: 0.7769330	total: 2m 6s	remaining: 8m 25s
550:	learn: 0.7786576	total: 2m 18s	remaining: 8m 11s
600:	learn: 0.7802117	total: 2m 31s	remaining: 7m 57s
650:	learn: 0.7817119	total: 2m 43s	remaining: 7m 44s
700:	learn: 0.7831382	total: 2m 55s	remaining: 7m 30s
750:	learn: 0.7846123	total: 3m 7s	remaining: 7m 17s
800:	learn: 0.7858425	total: 3m 20s	remaining: 7m 4s
850:	learn: 0.7871796	total: 3m 32s	remaining: 6m 51s
900:	learn: 0.7885721	total: 3m 45s	re

C:\Users\pavlo\anaconda3\lib\site-packages\catboost\core.py:2194: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.5130880	total: 628ms	remaining: 26m 9s
50:	learn: 0.7299934	total: 14.5s	remaining: 11m 34s
100:	learn: 0.7457412	total: 27.1s	remaining: 10m 43s
150:	learn: 0.7535157	total: 40.1s	remaining: 10m 23s
200:	learn: 0.7593634	total: 52.8s	remaining: 10m 3s
250:	learn: 0.7633628	total: 1m 5s	remaining: 9m 44s
300:	learn: 0.7666506	total: 1m 17s	remaining: 9m 27s
350:	learn: 0.7699505	total: 1m 30s	remaining: 9m 11s
400:	learn: 0.7728281	total: 1m 42s	remaining: 8m 56s
450:	learn: 0.7753223	total: 1m 54s	remaining: 8m 41s
500:	learn: 0.7768700	total: 2m 7s	remaining: 8m 27s
550:	learn: 0.7792425	total: 2m 19s	remaining: 8m 12s
600:	learn: 0.7806029	total: 2m 31s	remaining: 7m 59s
650:	learn: 0.7822143	total: 2m 44s	remaining: 7m 46s
700:	learn: 0.7837065	total: 2m 56s	remaining: 7m 33s
750:	learn: 0.7853864	total: 3m 8s	remaining: 7m 19s
800:	learn: 0.7867558	total: 3m 20s	remaining: 7m 6s
850:	learn: 0.7876918	total: 3m 33s	remaining: 6m 53s
900:	learn: 0.7892143	total: 3m 45s	r

C:\Users\pavlo\anaconda3\lib\site-packages\catboost\core.py:2194: UserWarning: Can't optimze method "evaluate" because self argument is used
  _check_train_params(params)


0:	learn: 0.5152621	total: 594ms	remaining: 24m 44s
50:	learn: 0.7289998	total: 14.1s	remaining: 11m 18s
100:	learn: 0.7463837	total: 26.8s	remaining: 10m 37s
150:	learn: 0.7551188	total: 39.4s	remaining: 10m 13s
200:	learn: 0.7606488	total: 52s	remaining: 9m 54s
250:	learn: 0.7657149	total: 1m 4s	remaining: 9m 35s
300:	learn: 0.7690799	total: 1m 16s	remaining: 9m 20s
350:	learn: 0.7718402	total: 1m 29s	remaining: 9m 6s
400:	learn: 0.7746455	total: 1m 41s	remaining: 8m 53s
450:	learn: 0.7766502	total: 1m 54s	remaining: 8m 39s
500:	learn: 0.7788593	total: 2m 6s	remaining: 8m 25s
550:	learn: 0.7804374	total: 2m 19s	remaining: 8m 12s
600:	learn: 0.7820231	total: 2m 31s	remaining: 7m 59s
650:	learn: 0.7832177	total: 2m 43s	remaining: 7m 45s
700:	learn: 0.7847705	total: 2m 56s	remaining: 7m 33s
750:	learn: 0.7862803	total: 3m 8s	remaining: 7m 19s
800:	learn: 0.7877741	total: 3m 20s	remaining: 7m 6s
850:	learn: 0.7888216	total: 3m 33s	remaining: 6m 53s
900:	learn: 0.7901850	total: 3m 45s	rem

In [16]:
import numpy as np
VER = 4
#l_CATBOOST = [(1,2), (3,4), (5,6), (7,8), (9,10), (11, 12), (12,13), (1,3), (2,4), (5, 7), (6, 8), (9,11), (10,12)]
#k = 2

l_CATBOOST = [(1,2,3,4,5,6,7,8,9,10,11,12,13)]
k = 13
CATBOOST_COUNTER = len(l_CATBOOST)
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87",
    #"D_62"#add
]
CATBOOST_COUNTER

1

In [ ]:
#with s_2
#change -1000
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87",
    "S_2"
     #"D_62"#
]

def func_calculate_v3(train_w_rnk, test_w_rnk, dfl, catboost_counter, ver, n_splits, random_state, category_cols, l_catboost,k):

     #params = {'task_type': 'GPU',
     #         'depth':[1,2,3,4,5,7,8,10], #глубина дерева
     #         'iterations':[1500,2000, 2500], # количество итераций 
     #         'learning_rate':[0.001, 0.01, 0.03,0.1,0.25, 0.5], #скорость обученя
     #         'l2_leaf_reg':[1,3,5,10,100], #коэфициент в регуляризации
     #         'bootstrap_type':['Bernoulli'], #как выбираем элементы  из выборки  #'Bernoulli', 'No' #нужно убрать temperature
     #         'border_count':[5,10,20,50,100,200],#скорость обучения GPU
     #             #'bagging_temperature':[0.03,0.25,0.75, 0.9], # коэффициенты с помощью которых ге
     #               #генерируются веса в баесовском бутстрепе 
     #             'random_strength':[0.2,0.5,0.8], #коэффициент участвующий при разделения дерева и оценки подсчета очков
     #             'max_ctr_complexity':[1,2,3,4,5], # количество листьев для категориальных признаков поиск переобучения 
     #         'thread_count': 4000, #4352 количество ядер на GPU
     #         'metric_period': 50, #каждые 50 итерацией вычисляется кастомная метрика
     #          'eval_metric': ['Logloss'], #AmexMetric() 
     #          'early_stopping_rounds': 50 #остановится если переобучилсиь
     #         }    
    params =  {'random_strength': 0.2,
    'border_count': 200,
    'bootstrap_type': 'Bernoulli',
    'eval_metric': AmexMetric(),
    'od_wait': 50,
    'iterations': 2500,
    'thread_count': 4000,
    'max_ctr_complexity': 1,
    'l2_leaf_reg': 1.25,
    #'loss_function': 'Logloss',
    'task_type': 'CPU',
    'depth': 2,
    'metric_period': 50,
    'learning_rate': 0.25}


    
    #test_blend = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID']
    test_stack = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID'].to_frame()
    
    #train_blend = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID']
    train_stack = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID'].to_frame()
    
    l_drop = ['rn']
    #l_0 = ['ID', 'target', 'customer_ID']
    
    l_drop_test = [ 'rn']
    #l_0_test = ['ID']
    #category_cols_all = []
    
    for rank,c in enumerate(l_catboost):
        l_0 = ['ID', 'target', 'customer_ID']
        l_0_test = ['ID']
        category_cols_all = [x+'_'+str(c[0]) for j1,x in enumerate(category_cols)]
        #category_cols_all = []
        dtrain = train_w_rnk[train_w_rnk.rn == c[0]]
        #print(dtrain.columns)
        dtrain.columns = [x+'_'+str(c[0]) for j,x in enumerate(dtrain.columns)]
        dtrain =  dtrain.rename(columns = {'ID_'+str(c[0]):'ID'})
        l_0.extend([x+'_'+str(c[0]) for j,x in enumerate(l_drop)])
        l_drop_all = l_0
        
        for j in range(1, k):
            dtrain_next = train_w_rnk[train_w_rnk.rn == c[j]]
            dtrain_next.columns = [x+'_'+str(c[j]) for j1,x in enumerate(dtrain_next.columns)]
            
            l_drop_add = [x+'_'+str(c[j]) for j1,x in enumerate(l_drop)]
            #print(l_drop_res)
            #print(l_drop_add)
            l_drop_all.extend(l_drop_add)
            
            category_cols_add = [x+'_'+str(c[j]) for j1,x in enumerate(category_cols)]
            category_cols_all.extend(category_cols_add)
            dtrain_next =  dtrain_next.rename(columns = {'ID_'+str(c[j]):'ID'})
            dtrain = dtrain.merge(dtrain_next, how = 'left', on = ['ID'])
        dft = dtrain     
        train_id = dft['ID']
     
        data = dft.merge(dfl, how = 'inner', on = 'ID')
        #dtrain =  dtrain.rename(columns = {'ID':'ID_1'})
        
        print(data.columns)
    
        #X_train = data.drop(columns = list(l_drop_all)).fillna(-1000) #fillna(method='ffill', axis = 1)
        X_train = data.drop(columns = list(l_drop_all)).fillna(method='ffill', axis = 1)
        y_train = data['target']
        
        #print(list(X_train.columns))
        #print(list(X_train.columns)[51])
        cat_dims = [X_train.columns.get_loc(i) for i in category_cols_all[:-1]]
        #print(category_cols_all)
        #print(cat_dims)
        
        for header in category_cols_all:
             X_train[header] = X_train[header].astype('category').cat.codes

        bestparams = params 
        clf = cb.CatBoostClassifier(**bestparams)
        print(train_stack)
      
        clf.fit(X_train, np.ravel(y_train), cat_features=cat_dims)
        clf.save_model(str(VER)+"_"+str(rank)+"_clf")
        print(str(VER)+"_"+str(rank)+': save model')
        
        del X_train
        del dtrain
        del dft
        del data

        dtest= test_w_rnk[test_w_rnk.rn == c[0]]
        dtest.columns = [x+'_'+str(c[0]) for j,x in enumerate(dtest.columns)]
        dtest =  dtest.rename(columns = {'ID_'+str(c[0]):'ID'})
        l_0_test.extend([x+'_'+str(c[0]) for j,x in enumerate(l_drop_test)])
        l_drop_all = l_0_test
        for j in range(1, k):
            dtest_next = test_w_rnk[test_w_rnk.rn == c[j]]
            dtest_next.columns = [x+'_'+str(c[j]) for j1,x in enumerate(dtest_next.columns)]
            
            l_drop_add = [x+'_'+str(c[j]) for j1,x in enumerate(l_drop_test)]
            l_drop_all.extend(l_drop_add)
            
            dtest_next =  dtest_next.rename(columns = {'ID_'+str(c[j]):'ID'})
            dtest = dtest.merge(dtest_next, how = 'left', on = ['ID'])
        
        
        #dftt = dtest
        test_id = dtest['ID']
        #dtest =  dtest.rename(columns = {'ID':'ID_1'})
        #dtest = dtest.drop(columns = list(l_drop_all)).fillna(-1000) #fillna(method='ffill', axis = 1)
        dtest = dftt.drop(columns = list(l_drop_all)).fillna(method='ffill', axis = 1)
        for header in category_cols_all:
             dtest[header] = dtest[header].astype('category').cat.codes

        res = clf.predict_proba(dtest)
        pred = pd.DataFrame({str(rank): res[:, 1].tolist(),})
        df_test = pd.concat([test_id.reset_index(drop = True), pred], axis = 1)
        test_stack = test_stack.merge(df_test, how = 'left', on = ['ID'])
        print(test_stack)
        test_stack.to_excel(str(VER)+'catboost_test.xlsx', index = False)
        train_stack.to_excel(str(VER)+'catboost_train.xlsx', index = False)
        print(str(VER)+"_"+str(rank)+': save result')
    return train_stack, test_stack 
      
train_stack, test_stack = func_calculate_v3(train_w_rnk, test_w_rnk, dfl, CATBOOST_COUNTER,
                                            VER, FOLD, SEED, cat_features, l_CATBOOST, k)

Index(['S_2_1', 'P_2_1', 'D_39_1', 'B_1_1', 'B_2_1', 'R_1_1', 'S_3_1',
       'D_41_1', 'B_3_1', 'D_42_1',
       ...
       'D_139_13', 'D_140_13', 'D_141_13', 'D_142_13', 'D_143_13', 'D_144_13',
       'D_145_13', 'rn_13', 'customer_ID', 'target'],
      dtype='object', length=2473)


In [19]:
#with s_2
#change -1000
from catboost import CatBoostClassifier
cat_features = [
    "B_30",
    "B_31",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68",
    "D_87",
    #"S_2"
     #"D_62"#
]

def func_calculate_v4(train_w_rnk, test_w_rnk, dfl, catboost_counter, ver, n_splits, random_state, category_cols, l_catboost,k):

     #params = {'task_type': 'GPU',
     #         'depth':[1,2,3,4,5,7,8,10], #глубина дерева
     #         'iterations':[1500,2000, 2500], # количество итераций 
     #         'learning_rate':[0.001, 0.01, 0.03,0.1,0.25, 0.5], #скорость обученя
     #         'l2_leaf_reg':[1,3,5,10,100], #коэфициент в регуляризации
     #         'bootstrap_type':['Bernoulli'], #как выбираем элементы  из выборки  #'Bernoulli', 'No' #нужно убрать temperature
     #         'border_count':[5,10,20,50,100,200],#скорость обучения GPU
     #             #'bagging_temperature':[0.03,0.25,0.75, 0.9], # коэффициенты с помощью которых ге
     #               #генерируются веса в баесовском бутстрепе 
     #             'random_strength':[0.2,0.5,0.8], #коэффициент участвующий при разделения дерева и оценки подсчета очков
     #             'max_ctr_complexity':[1,2,3,4,5], # количество листьев для категориальных признаков поиск переобучения 
     #         'thread_count': 4000, #4352 количество ядер на GPU
     #         'metric_period': 50, #каждые 50 итерацией вычисляется кастомная метрика
     #          'eval_metric': ['Logloss'], #AmexMetric() 
     #          'early_stopping_rounds': 50 #остановится если переобучилсиь
     #         }    
    params =  {'random_strength': 0.2,
    'border_count': 200,
    'bootstrap_type': 'Bernoulli',
    'eval_metric': AmexMetric(),
    'od_wait': 50,
    'iterations': 2500,
    'thread_count': 4000,
    'max_ctr_complexity': 1,
    'l2_leaf_reg': 1.25,
    #'loss_function': 'Logloss',
    'task_type': 'CPU',
    'depth': 2,
    'metric_period': 50,
    'learning_rate': 0.25}


    
    #test_blend = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID']
    test_stack = test_w_rnk.loc[test_w_rnk.rn == 1, 'ID'].to_frame()
    
    #train_blend = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID']
    train_stack = train_w_rnk.loc[train_w_rnk.rn == 1, 'ID'].to_frame()
    
    l_drop = ['rn', 'S_2']
    #l_0 = ['ID', 'target', 'customer_ID']
    
    l_drop_test = [ 'rn','S_2']
    #l_0_test = ['ID']
    #category_cols_all = []
    
    for rank,c in enumerate(l_catboost):
        l_0 = ['ID', 'target', 'customer_ID']
        l_0_test = ['ID']
        category_cols_all = [x+'_'+str(c[0]) for j1,x in enumerate(category_cols)]
        for j in range(1, k):
            category_cols_add = [x+'_'+str(c[j]) for j1,x in enumerate(category_cols)]
            category_cols_all.extend(category_cols_add)
        
      
        dtest= test_w_rnk[test_w_rnk.rn == c[0]]
        dtest.columns = [x+'_'+str(c[0]) for j,x in enumerate(dtest.columns)]
        dtest =  dtest.rename(columns = {'ID_'+str(c[0]):'ID'})
        l_0_test.extend([x+'_'+str(c[0]) for j,x in enumerate(l_drop_test)])
        l_drop_all = l_0_test
        for j in range(1, k):
            dtest_next = test_w_rnk[test_w_rnk.rn == c[j]]
            dtest_next.columns = [x+'_'+str(c[j]) for j1,x in enumerate(dtest_next.columns)]
            
            l_drop_add = [x+'_'+str(c[j]) for j1,x in enumerate(l_drop_test)]
            l_drop_all.extend(l_drop_add)
            
            dtest_next =  dtest_next.rename(columns = {'ID_'+str(c[j]):'ID'})
            dtest = dtest.merge(dtest_next, how = 'left', on = ['ID'])
        
        
        #dftt = dtest
        test_id = dtest['ID']
        #dtest =  dtest.rename(columns = {'ID':'ID_1'})
        dtest = dtest.drop(columns = list(l_drop_all)).fillna(-1000) #fillna(method='ffill', axis = 1)
        #X_test = dftt.drop(columns = list(l_drop_all)).fillna(method='ffill', axis = 1)
        for header in category_cols_all:
             dtest[header] = dtest[header].astype('category').cat.codes
                
       
        from_file = CatBoostClassifier()

        clf = from_file.load_model("4_0_clf")
        res = clf.predict_proba(dtest)
        pred = pd.DataFrame({str(rank): res[:, 1].tolist(),})
        df_test = pd.concat([test_id.reset_index(drop = True), pred], axis = 1)
        test_stack = test_stack.merge(df_test, how = 'left', on = ['ID'])
        print(test_stack)
        test_stack.to_excel(str(VER)+'catboost_test.xlsx', index = False)
        train_stack.to_excel(str(VER)+'catboost_train.xlsx', index = False)
        print(str(VER)+"_"+str(rank)+': save result')
    return  test_stack 
      
test_stack = func_calculate_v4(train_w_rnk, test_w_rnk, dfl, CATBOOST_COUNTER,
                                            VER, FOLD, SEED, cat_features, l_CATBOOST, k)

                          ID         0
0                63146778244  0.022199
1              4712478813452  0.052601
2             10045695118710  0.712037
3             24584914086599  0.596756
4             25718247626061  0.001329
...                      ...       ...
924616  18446691153987313650  0.001532
924617  18446702931748678034  0.001420
924618  18446710860663557019  0.135841
924619  18446711164465817979  0.006279
924620  18446738444797682302  0.003263

[924621 rows x 2 columns]
4_0: save result
